In [1]:
#!/usr/bin/env python3
"""
Single-Agent, Multi-Task Demo (Planning • Strategy • Autonomy)
- One agent processes a queue of natural-language tasks/goals sequentially.
- Each task is handled end-to-end (plan → strategy → schedule → forecast → brief).
- Shows: single decision-maker, multiple tasks, lightweight tool-use, and memory per task.

Run examples:
    python single_agent_multi_task.py \
        --goal "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales." \
        --goal "Create a 2-day influencer sprint for launch with ₹20,000 budget." \
        --brand FleetAthlete --theme Speed

    # Or load from a JSON file containing a list of goal strings:
    python single_agent_multi_task.py --goals_file goals.json

Outputs:
- Console summary for each task
- A combined report saved as multi_task_report.txt
"""
from __future__ import annotations
import argparse
import json
import math
import random
import re
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional

In [2]:


# -----------------------------
# "Tools" the agent can use
# -----------------------------

class CalculatorTool:
    """Tiny safe-ish calculator for simple expressions like '50000*0.25'."""
    ALLOWED = set("0123456789.+-*/()% ")

    def run(self, expr: str) -> float:
        if not set(expr) <= self.ALLOWED:
            raise ValueError("Unsafe expression.")
        return float(eval(expr, {"__builtins__": None}, {}))

class KnowledgeBaseTool:
    """Mocked 'research' tool with pragmatic defaults the agent can consult."""
    KB = {
        "cpc_india_sports_paid_social": 6.5,       # ₹ per click (rough heuristic)
        "ctr_organic_social": 0.015,               # 1.5% click-through
        "conv_rate_ecom": 0.02,                    # 2% site conversion
        "avg_order_value": 1800.0,                 # ₹
        "influencer_micro_cost_per_post": 5000.0,  # ₹
        "email_ctr": 0.03,
        "email_conv": 0.02,
        "best_posting_times": ["7:30 AM", "12:30 PM", "7:30 PM"],
        "channels": ["Paid Social", "Organic Social", "Influencers", "Email/SMS", "Content/Blog"],
    }

    def lookup(self, key: str) -> Any:
        return self.KB.get(key)

class NoteTool:
    """Simple in-run memory (resets each run_many call)."""
    def __init__(self):
        self.memory: Dict[str, Any] = {}

    def set(self, k: str, v: Any):
        self.memory[k] = v

    def get(self, k: str, default=None):
        return self.memory.get(k, default)

    def add_list(self, k: str, v: Any):
        self.memory.setdefault(k, []).append(v)

class SchedulerTool:
    def make_schedule(self, days: int, slots: Optional[List[str]] = None) -> List[Dict[str, Any]]:
        if slots is None:
            slots = ["Morning", "Afternoon", "Evening"]
        plan = []
        for d in range(1, max(1, days) + 1):
            plan.append({
                "day": d,
                "slots": {s: [] for s in slots}
            })
        return plan

class IdeaTool:
    def brainstorm_posts(self, brand: str, theme: str, n: int = 5) -> List[str]:
        random.seed(42 + len(brand) + len(theme))
        verbs = ["Crush", "Chase", "Own", "Level-Up", "Ignite", "Unlock"]
        angles = ["form tips", "micro-workouts", "gear care", "coach wisdom", "community stories", "before/after"]
        ideas = []
        for _ in range(n * 2):  # extra draws to increase variety
            v = random.choice(verbs)
            a = random.choice(angles)
            ideas.append(f"{v} your game: {a} ({theme} × {brand})")
        # de-dup while preserving order
        seen, out = set(), []
        for i in ideas:
            if i not in seen:
                seen.add(i)
                out.append(i)
            if len(out) == n:
                break
        return out

In [3]:
# -----------------------------
# Planner & Strategy Modules
# -----------------------------

@dataclass
class Step:
    name: str
    done: bool = False
    details: Dict[str, Any] = field(default_factory=dict)

class Planner:
    BUDGET_REGEX = r"(₹|INR|\$)?\s?([0-9][0-9,\.]*)"
    DAYS_REGEX = r"(\d+)\s*(day|days|d)\b"

    def parse_budget(self, text: str) -> Optional[float]:
        matches = re.findall(self.BUDGET_REGEX, text, flags=re.I)
        nums = []
        for _, raw in matches:
            try:
                nums.append(float(raw.replace(",", "")))
            except:
                pass
        return max(nums) if nums else None

    def parse_days(self, text: str) -> Optional[int]:
        m = re.search(self.DAYS_REGEX, text, flags=re.I)
        return int(m.group(1)) if m else None

    def plan(self, goal: str) -> Dict[str, Any]:
        budget = self.parse_budget(goal)
        days = self.parse_days(goal)
        steps = [
            Step("Clarify constraints", details={"need": ["budget", "days"]}),
            Step("Choose channels and split budget"),
            Step("Create content ideas"),
            Step("Build schedule & checklist"),
            Step("Forecast outcomes"),
            Step("Assemble final brief"),
        ]
        return {"budget": budget, "days": days, "steps": steps}

class Strategist:
    def __init__(self, kb: KnowledgeBaseTool, calc: CalculatorTool):
        self.kb = kb
        self.calc = calc

    def choose_channels(self, budget: float, days: int) -> Dict[str, Any]:
        if budget < 20000:
            alloc = {"Organic Social": 0.0, "Influencers": 0.6, "Content/Blog": 0.2, "Email/SMS": 0.2}
        elif budget < 100000:
            alloc = {"Paid Social": 0.5, "Influencers": 0.2, "Organic Social": 0.15, "Email/SMS": 0.15}
        else:
            alloc = {"Paid Social": 0.6, "Influencers": 0.2, "Content/Blog": 0.1, "Email/SMS": 0.1}
        split = {k: round(v, 2) for k, v in alloc.items()}
        rupees = {k: round(budget * v) for k, v in split.items()}
        return {"allocation": split, "rupees": rupees, "days": days}

    def simple_forecast(self, rupees: Dict[str, float]) -> Dict[str, Any]:
        cpc = self.kb.lookup("cpc_india_sports_paid_social")
        conv = self.kb.lookup("conv_rate_ecom")
        aov = self.kb.lookup("avg_order_value")
        influencer_cost = self.kb.lookup("influencer_micro_cost_per_post")

        results: Dict[str, Dict[str, Any]] = {}

        if "Paid Social" in rupees and rupees["Paid Social"] > 0 and cpc:
            clicks = rupees["Paid Social"] / cpc
            orders = clicks * conv
            rev = orders * aov
            results["Paid Social"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        if "Influencers" in rupees and rupees["Influencers"] > 0 and influencer_cost:
            posts = max(1, int(rupees["Influencers"] // influencer_cost))
            clicks = posts * 150
            orders = clicks * conv
            rev = orders * aov
            results["Influencers"] = {"posts": posts, "clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        if "Organic Social" in rupees:
            clicks = 400
            orders = clicks * conv
            results["Organic Social"] = {"clicks": clicks, "orders": int(orders)}

        if "Email/SMS" in rupees:
            list_size = 10000
            ctr = self.kb.lookup("email_ctr")
            econv = self.kb.lookup("email_conv")
            clicks = list_size * ctr
            orders = clicks * econv
            rev = orders * aov
            results["Email/SMS"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        totals = {
            "orders": sum(v.get("orders", 0) for v in results.values()),
            "revenue": sum(v.get("revenue", 0) for v in results.values())
        }
        results["TOTALS"] = totals
        return results


In [4]:
# -----------------------------
# Tasks & Agent
# -----------------------------

@dataclass
class Task:
    id: int
    goal: str
    status: str = "queued"  # queued → running → done/failed
    result: Optional["AgentOutput"] = None
    error: Optional[str] = None

@dataclass
class AgentOutput:
    goal: str
    constraints: Dict[str, Any]
    strategy: Dict[str, Any]
    schedule: List[Dict[str, Any]]
    ideas: List[str]
    forecast: Dict[str, Any]
    brief: str

class SingleAgent:
    def __init__(self):
        self.calc = CalculatorTool()
        self.kb = KnowledgeBaseTool()
        self.scheduler = SchedulerTool()
        self.ideas = IdeaTool()
        self.planner = Planner()
        self.strategist = Strategist(self.kb, self.calc)
        # fresh NoteTool per run_many() call

    # -------- single-task run --------
    def run_one(self, goal: str, brand: str = "YourSportsBrand", theme: str = "Performance") -> AgentOutput:
        notes = NoteTool()
        plan = self.planner.plan(goal)
        budget, days = plan["budget"], plan["days"]

        if budget is None:
            budget = 50000.0
        if days is None:
            days = 3

        notes.set("constraints", {"budget": budget, "days": days})

        strat = self.strategist.choose_channels(budget, days)
        notes.set("strategy", strat)

        content_ideas = self.ideas.brainstorm_posts(brand, theme, n=6)
        notes.set("ideas", content_ideas)

        sched = self.scheduler.make_schedule(days, slots=self.kb.lookup("best_posting_times"))
        for day in sched:
            for t in day["slots"]:
                day["slots"][t] = [
                    f"Post: {random.choice(content_ideas)}",
                    "Story/Reel: behind-the-scenes",
                ]
        notes.set("schedule", sched)

        forecast = self.strategist.simple_forecast(strat["rupees"])
        notes.set("forecast", forecast)

        brief = self._assemble_brief(goal, brand, theme, notes.get("constraints"), strat, sched, content_ideas, forecast)

        return AgentOutput(
            goal=goal,
            constraints=notes.get("constraints"),
            strategy=strat,
            schedule=sched,
            ideas=content_ideas,
            forecast=forecast,
            brief=brief,
        )

    def _assemble_brief(self, goal, brand, theme, constraints, strat, sched, ideas, forecast) -> str:
        lines: List[str] = []
        lines.append(f"# Launch Brief\n")
        lines.append(f"**Goal:** {goal}")
        lines.append(f"**Brand/Theme:** {brand} / {theme}")
        lines.append(f"**Constraints:** Budget ₹{int(constraints['budget'])}, Duration {constraints['days']} days\n")

        lines.append("## Channel Strategy & Budget Split")
        for k, v in strat["allocation"].items():
            lines.append(f"- {k}: {int(v*100)}% (₹{strat['rupees'][k]})")

        lines.append("\n## Content Ideas (shortlist)")
        for i in ideas:
            lines.append(f"- {i}")

        lines.append("\n## Day-by-Day Schedule")
        for day in sched:
            lines.append(f"- **Day {day['day']}**")
            for slot, tasks in day["slots"].items():
                for task in tasks:
                    lines.append(f"  - {slot}: {task}")

        lines.append("\n## Rough Forecast (very rough, for planning only)")
        for ch, vals in forecast.items():
            if ch == "TOTALS":
                continue
            pretty = ", ".join(f"{k}={v}" for k, v in vals.items())
            lines.append(f"- {ch}: {pretty}")
        totals = forecast.get("TOTALS", {})
        lines.append(f"- **TOTALS:** orders={totals.get('orders', 0)}, revenue=₹{totals.get('revenue', 0)}")

        lines.append("\n## Risks & Mitigations")
        lines.append("- Creative fatigue → rotate formats daily; reuse top winners only.")
        lines.append("- Tracking issues → verify UTM and pixels before Day 1.")
        lines.append("- Inventory runouts → sync with ops; set real scarcity only.")

        lines.append("\n## Success Metrics")
        lines.append("- CTR, CPC (paid), follower growth (organic), email CTR/conv, total orders & revenue.")

        return "\n".join(lines)

    # -------- multi-task orchestration --------
    def run_many(self, goals: List[str], brand: str = "YourSportsBrand", theme: str = "Performance") -> List[Task]:
        tasks: List[Task] = [Task(id=i+1, goal=g) for i, g in enumerate(goals)]
        for t in tasks:
            t.status = "running"
            try:
                result = self.run_one(t.goal, brand=brand, theme=theme)
                t.result = result
                t.status = "done"
            except Exception as e:
                t.status = "failed"
                t.error = str(e)
        return tasks



In [5]:
# -----------------------------
# CLI
# -----------------------------

# Removed argument parsing and main execution block for Colab compatibility

def run_agent_in_colab(goals: List[str], brand: str = "YourSportsBrand", theme: str = "Performance") -> List[Task]:
    """Helper function to run the SingleAgent in a Colab environment."""
    agent = SingleAgent()
    tasks = agent.run_many(goals, brand=brand, theme=theme)

    # Console summary
    for t in tasks:
        print(f"=== Task #{t.id} | {t.status.upper()} ===")
        print(t.goal)
        if t.status == "done" and t.result:
            totals = t.result.forecast.get("TOTALS", {})
            print("Constraints:", t.result.constraints)
            print("Totals:", totals)
        elif t.error:
            print("Error:", t.error)
        print()

    # Write combined report
    report_path = "multi_task_report.txt"
    write_report(tasks, path=report_path)
    print(f"Combined report written to: {report_path}")

    return tasks


def write_report(tasks: List[Task], path: str = "multi_task_report.txt") -> str:
    lines: List[str] = []
    lines.append("=== SINGLE AGENT • MULTI TASK REPORT ===\n")
    for t in tasks:
        lines.append(f"Task #{t.id}: {t.goal}")
        lines.append(f"Status: {t.status}\n")
        if t.status == "done" and t.result:
            r = t.result
            lines.append(f"Constraints: {r.constraints}")
            lines.append(f"Strategy: {json.dumps(r.strategy, indent=2)}")
            lines.append(f"Top Ideas: {r.ideas[:3]}")
            lines.append(f"Forecast Totals: {r.forecast.get('TOTALS', {})}\n")
            lines.append(r.brief)
            lines.append("\n" + ("-" * 60) + "\n")
        elif t.error:
            lines.append(f"Error: {t.error}\n")
    with open(path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    return path

In [6]:
# Example usage:
goals = [
    "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales.",
    "Create a 2-day influencer sprint for launch with ₹20,000 budget."
]

run_agent_in_colab(goals, brand="FleetAthlete", theme="Speed")

=== Task #1 | DONE ===
Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales.
Constraints: {'budget': 50000.0, 'days': 3}
Totals: {'orders': 96, 'revenue': 160061}

=== Task #2 | DONE ===
Create a 2-day influencer sprint for launch with ₹20,000 budget.
Constraints: {'budget': 20000.0, 'days': 3}
Totals: {'orders': 47, 'revenue': 71584}

Combined report written to: multi_task_report.txt


[Task(id=1, goal='Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales.', status='done', result=AgentOutput(goal='Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales.', constraints={'budget': 50000.0, 'days': 3}, strategy={'allocation': {'Paid Social': 0.5, 'Influencers': 0.2, 'Organic Social': 0.15, 'Email/SMS': 0.15}, 'rupees': {'Paid Social': 25000, 'Influencers': 10000, 'Organic Social': 7500, 'Email/SMS': 7500}, 'days': 3}, schedule=[{'day': 1, 'slots': {'7:30 AM': ['Post: Chase your game: form tips (Speed × FleetAthlete)', 'Story/Reel: behind-the-scenes'], '12:30 PM': ['Post: Chase your game: form tips (Speed × FleetAthlete)', 'Story/Reel: behind-the-scenes'], '7:30 PM': ['Post: Crush your game: micro-workouts (Speed × FleetAthlete)', 'Story/Reel: behind-the-scenes']}}, {'day': 2, 'slots': {'7:30 AM': ['Post: Ignite your game: form tips (Speed × FleetAthlete)', 'Story/Reel: behind-the-scenes'], '12:30 PM':